In [60]:
#Import libraries and set standars for visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
import re
import itertools
import collections
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from stop_words import get_stop_words
import pandas as pd
import glob
from itertools import *

###################################

In [61]:
def stack_files():
    globbed_files = glob.glob("russian_*.csv") #creates a list of all csv files

    data = [] # pd.concat takes a list of dataframes as an agrument
    for csv in globbed_files:
        frame = pd.read_csv(csv)
        data.append(frame)

    global bigframe
    bigframe = pd.concat(data, ignore_index=True) #dont want pandas to try an align row indexes
    return bigframe

stack_files()

,author,hashtags,translated,language,date_inputted,entities,type,model
0,CPBN,РаботайтеБратья,WorkBrethren,ru,2021-03-31,NaN,NaN,NaN
1,belamova,stopBTRC,StopBtrc,ru,2021-03-31,NaN,NaN,NaN
2,👤,бан,ban,ru,2021-03-31,NaN,NaN,NaN
3,San,КарысныяСпасылкі,KarysnyyasPasylki,ru,2021-03-31,NaN,NaN,NaN
4,belamova,Жодино,Zhodino,ru,2021-03-31,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2604,Сергей,NaN,DAILY,ru,2021-03-31,ЕЖЕДНЕВНЫЕ,ORG,ru_core_news_sm
2605,Сергей,NaN,VITEBSK,ru,2021-03-31,ВИТЕБСК,ORG,ru_core_news_sm
2606,Сергей,NaN,Orsha Borisov Baranovichi Ivatsevich Pinsk Best,ru,2021-03-31,Орша Борисов Барановичи Ивацевич Пинск Лучшие,PER,ru_core_news_sm
2607,Сергей,NaN,Nos,ru,2021-03-31,Номера,PER,ru_core_news_sm


In [63]:
import re
bigframe['entities'] =  [re.sub('@[^\s]+','', str(x)) for x in bigframe['entities']]
bigframe['entities'] =  [re.sub('((www\.[^\s]+)|(https?://[^\s]+))','', str(x)) for x in bigframe['entities']]# remove all the links
bigframe['entities'] =  [re.sub(r'[^\w]', ' ', str(x)) for x in bigframe['entities']] 
bigframe['entities'] =  [re.sub(r'\d+', ' ', str(x)) for x in bigframe['entities']] # remove digits
bigframe['entities'] =  [re.sub('RT', ' ', str(x)) for x in bigframe['entities']] # remove RT
bigframe['entities'] = [x.lower() for x in bigframe['entities']] #lowercase everything
bigframe['author'] =  [re.sub('@[^\s]+','', str(x)) for x in bigframe['author']]
bigframe['author'] =  [re.sub('((www\.[^\s]+)|(https?://[^\s]+))','', str(x)) for x in bigframe['author']]# remove all the links
bigframe['author'] =  [re.sub(r'[^\w]', ' ', str(x)) for x in bigframe['author']] 
bigframe['author'] =  [re.sub(r'\d+', ' ', str(x)) for x in bigframe['author']] # remove digits
bigframe['author'] =  [re.sub('RT', ' ', str(x)) for x in bigframe['author']] # remove RT
bigframe['author'] = [x.lower() for x in bigframe['author']] #lowercase everything
#bigframe['content'] = [' '.join([y for y in x.split() if y not in stopwords_lst]) for x in bigframe['content']] #remove stopwords
bigframe

#new_bf = pd.concat([pd.Series(row['author'], row['content'].split(' '))
           #for _, row in bigframe.iterrows()]).reset_index()
#new_bf

,author,hashtags,translated,language,date_inputted,entities,type,model
0,cpbn,РаботайтеБратья,WorkBrethren,ru,2021-03-31,nan,NaN,NaN
1,belamova,stopBTRC,StopBtrc,ru,2021-03-31,nan,NaN,NaN
2,,бан,ban,ru,2021-03-31,nan,NaN,NaN
3,san,КарысныяСпасылкі,KarysnyyasPasylki,ru,2021-03-31,nan,NaN,NaN
4,belamova,Жодино,Zhodino,ru,2021-03-31,nan,NaN,NaN
...,...,...,...,...,...,...,...,...
2604,сергей,NaN,DAILY,ru,2021-03-31,ежедневные,ORG,ru_core_news_sm
2605,сергей,NaN,VITEBSK,ru,2021-03-31,витебск,ORG,ru_core_news_sm
2606,сергей,NaN,Orsha Borisov Baranovichi Ivatsevich Pinsk Best,ru,2021-03-31,орша борисов барановичи ивацевич пинск лучшие,PER,ru_core_news_sm
2607,сергей,NaN,Nos,ru,2021-03-31,номера,PER,ru_core_news_sm


In [64]:
def edge_list():
    #filtered = bigframe[bigframe['parentid'].isin(bigframe['author'].unique())]
    #result = filtered.rename(columns={'author': 'Source', 'entities': 'Target'}) \
    result = bigframe.rename(columns={'author': 'Source', 'entities': 'Target'}) \
                 .groupby(['Source', 'Target']) \
                 .size() \
                 .reset_index(name='Weight')
    result.to_csv('Telegram_Edgelist.csv')
    return result

edge_list()

,Source,Target,Weight
0,,nan,1
1,,беларусь,1
2,,зато,1
3,,минск,1
4,,пишите,1
...,...,...,...
2537,смелым перамога,карпенков,1
2538,смелым перамога,слева,1
2539,юрий,динамо,1
2540,юрий,ниной багинской,1


In [4]:
import re
bigframe['content'] =  [re.sub('@[^\s]+','', str(x)) for x in bigframe['content']]
bigframe['content'] =  [re.sub('((www\.[^\s]+)|(https?://[^\s]+))','', str(x)) for x in bigframe['content']]# remove all the links
bigframe['content'] =  [re.sub(r'[^\w]', ' ', str(x)) for x in bigframe['content']] 
bigframe['content'] =  [re.sub(r'\d+', ' ', str(x)) for x in bigframe['content']] # remove digits
bigframe['content'] =  [re.sub('RT', ' ', str(x)) for x in bigframe['content']] # remove RT
bigframe['content'] = [x.lower() for x in bigframe['content']] #lowercase everything
bigframe['author'] =  [re.sub('@[^\s]+','', str(x)) for x in bigframe['author']]
bigframe['author'] =  [re.sub('((www\.[^\s]+)|(https?://[^\s]+))','', str(x)) for x in bigframe['author']]# remove all the links
bigframe['author'] =  [re.sub(r'[^\w]', ' ', str(x)) for x in bigframe['author']] 
bigframe['author'] =  [re.sub(r'\d+', ' ', str(x)) for x in bigframe['author']] # remove digits
bigframe['author'] =  [re.sub('RT', ' ', str(x)) for x in bigframe['author']] # remove RT
bigframe['author'] = [x.lower() for x in bigframe['author']] #lowercase everything
#bigframe['content'] = [' '.join([y for y in x.split() if y not in stopwords_lst]) for x in bigframe['content']] #remove stopwords
bigframe

#new_bf = pd.concat([pd.Series(row['author'], row['content'].split(' '))
           #for _, row in bigframe.iterrows()]).reset_index()
#new_bf

,contentid,content,authoreddate,language,permalink,platform,author,doc_type,parentid,num_followers,num_posts,project
0,1a28d7b5-bf7a-52ec-8d40-009075ecafad,мусора позорники,2021-03-26 03:44:53,NaN,telegram.org,telegram,володя яблоков,telegram_stream_message,chatpartizan,NaN,NaN,BADGER_G
1,41e9c56f-3051-5402-8140-bb4fdebeb812,если уж беларусы не могут собраться то что гов...,2021-03-26 03:44:38,NaN,telegram.org,telegram,regular reparereo,telegram_stream_message,chatpartizan,NaN,NaN,BADGER_G
2,a2f186a9-f1cd-5425-ba90-5401b33b3855,лукашенко пидор старый,2021-03-26 03:43:51,NaN,telegram.org,telegram,володя яблоков,telegram_stream_message,chatpartizan,NaN,NaN,BADGER_G
3,7a06c198-3ac7-5006-a263-8d85793301f3,санта клаус херов,2021-03-26 03:43:00,NaN,telegram.org,telegram,володя яблоков,telegram_stream_message,chatpartizan,NaN,NaN,BADGER_G
4,eaba1fa2-1154-5880-8248-8291060d8ea3,відэа сілавікі выбіваюць дзверы ў дом у брыле...,2021-03-26 02:49:02,NaN,telegram.org,telegram,cpbn,telegram_stream_message,chatpartizan,NaN,NaN,BADGER_G
...,...,...,...,...,...,...,...,...,...,...,...,...
2247,bd084049-1df6-5cec-9cfa-d4cc6855fe6d,расстрелять пидараса,2021-03-29 21:53:56,NaN,telegram.org,telegram,ales gaaga,telegram_stream_message,chatpartizan,NaN,NaN,BADGER_G
2248,0f3acb6a-0689-5b22-9aa8-128d6d673f65,страна действительно должна знать этих пид,2021-03-29 21:31:11,NaN,telegram.org,telegram,vint,telegram_stream_message,chatpartizan,NaN,NaN,BADGER_G
2249,55b961b9-fc92-5521-ab3c-7167e06cb9d2,ты бы хоть аву поменял что ли,2021-03-29 21:27:50,NaN,telegram.org,telegram,group,telegram_stream_message,chatpartizan,NaN,NaN,BADGER_G
2250,e7c7600b-7251-5698-b67e-70b347bc34dc,ахахахахах трехпроцентные они такие трехпроцен...,2021-03-29 21:23:18,NaN,telegram.org,telegram,єнот,telegram_stream_message,chatpartizan,NaN,NaN,BADGER_G


In [5]:
#new_bf = pd.concat([pd.Series(row['author'], row['content'].split(' '))
#           for _, row in bigframe.iterrows()])
new_bf = pd.DataFrame(columns = ['author', 'content'])
new_bf['author'] = bigframe.author.copy()
new_bf['content'] = bigframe.content.copy()
#new_bf['content_list'] = new_bf['content'].str.split(" ")
new_bf = new_bf[['author','content_list']]

#new_bf = new_bf.apply(pd.Series.explode)


In [6]:
new_bf = new_bf.groupby(['author', 'content_list']).size().reset_index(name='Weight').rename(columns={"author": "Source", "content_list": "Target"})


,Source,Target,Weight
0,,,131
1,,p,2
2,,s,2
3,,telegram,1
4,,а,1
...,...,...,...
23205,哈哈,почему,2
23206,哈哈,почте,2
23207,哈哈,про,2
23208,哈哈,удалили,2


In [35]:
a = pd.get_dummies(new_bf, prefix='', prefix_sep='', columns = ['Target'])
a

,Source,Weight,,a,abvièščańnia,accountability,adbyłòsia,adin,adradžennie,advàžny,...,ўла,ўнутры,ўпершыню,ўсе,ўстаўная,ўся,ўсяго,ўсё,ўсім,ґра
0,,131,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23205,哈哈,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23206,哈哈,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23207,哈哈,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23208,哈哈,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# get unique Target values
target = list(new_bf['Target'].unique())
total_dfs = []
for word in target:
    d = {}
    temp = new_bf.loc[new_bf['Target'] == word]
    d['word'] = word
    d['df']   = temp
    d['authors'] = list(temp.Source.unique())
    total_dfs.append(d)
    

In [38]:
edges = pd.DataFrame(columns = ['Source','Target'])
edges['Source'] = [obj['word'] for obj in total_dfs]
edges['Target'] = [obj['authors'] for obj in total_dfs]
edges = edges.apply(pd.Series.explode)
edges

,Source,Target
0,,
0,,
0,,
0,,
0,,a
...,...,...
12653,назаўжды,іншая
12654,разница,ㅤㅤㅤ
12655,щас,ㅤㅤㅤ
12656,найдена,哈哈


In [10]:
new_bf.pivot_table(index=['Source'], columns='Target')

Weight                   \
Target                                             a abvièščańnia   
Source                                                              
                                           131.0 NaN          NaN   
                                            22.0 NaN          NaN   
                                            20.0 NaN          NaN   
                                             8.0 NaN          NaN   
    outcast bro     pr   лапа  ну ёба на     NaN NaN          NaN   
...                                          ...  ..          ...   
єнот                                         NaN NaN          NaN   
і                                           28.0 NaN          NaN   
іншая                                        8.0 NaN          NaN   
ㅤㅤㅤ                                          4.0 NaN          NaN   
哈哈                                           4.0 NaN          NaN   

                                                                         \
Target                                    accountability adbyłòsia adin   
Source                                                                    
                                                     NaN       NaN  NaN   
                                                     NaN       NaN  NaN   
                                                     NaN       NaN  NaN   
                                                     NaN       NaN  NaN   
    outcast bro     pr   лапа  ну ёба на             NaN       NaN  NaN   
...                                                  ...       ...  ...   
єнот                                                 NaN       NaN  NaN   
і                                                    NaN       NaN  NaN   
іншая                                                NaN       NaN  NaN   
ㅤㅤㅤ                                                  NaN       NaN  NaN   
哈哈                                                   NaN       NaN  NaN   

                                                                               \
Target                                    adradžennie advàžny adznàku aftrade   
Source                                                                          
                                                  NaN     NaN     NaN     NaN   
                                                  NaN     NaN     NaN     NaN   
                                                  NaN     NaN     NaN     NaN   
                                                  NaN     NaN     NaN     NaN   
    outcast bro     pr   лапа  ну ёба на          NaN     NaN     NaN     NaN   
...                                               ...     ...     ...     ...   
єнот                                              NaN     NaN     NaN     NaN   
і                                                 NaN     NaN     NaN     NaN   
іншая                                             NaN     NaN     NaN     NaN   
ㅤㅤㅤ                                               NaN     NaN     NaN     NaN   
哈哈                                                NaN     NaN     NaN     NaN   

                                           ...                          \
Target                                     ... ўла ўнутры ўпершыню ўсе   
Source                                     ...                           
                                           ... NaN    NaN      NaN NaN   
                                           ... NaN    NaN      NaN NaN   
                                           ... NaN    NaN      NaN NaN   
                                           ... NaN    NaN      NaN NaN   
    outcast bro     pr   лапа  ну ёба на   ... NaN    NaN      NaN NaN   
...                                        ...  ..    ...      ...  ..   
єнот                                       ... NaN    NaN      NaN NaN   
і                                          ... NaN    NaN      NaN NaN   
іншая                                      ... NaN    NaN      NaN NaN   
ㅤ

In [56]:
df1 = pd.DataFrame(data=list(combinations(uniq_nodes, 2)),  columns=['Source', 'Target'])

In [48]:
def edge_list_words():
    filtered = df[df['author'].isin(df['words'].unique())]
    result = df1.groupby(['Source', 'Target']) \
                 .size() \
                 .reset_index(name='Weight')
    result.to_csv('Telegram_Edgelist_semantic.csv')
    return result

edge_list_words()

,Source,Target,Weight
0,***,1,1
1,***,564,1
2,***,96049,1
3,***,A,1
4,***,A R,1
...,...,...,...
37945,😘,Че,1
37946,😘,Юра,1
37947,😘,остап ибрагимович,1
37948,😘,"✳️- Outcast BrO -✳️ pr0💯 Лапа, ну ёба-на🔥",1


In [7]:
new_bf

,author,content_list,Weight
0,***,,20
1,***,алексеев,2
2,***,без,2
3,***,беларусь,2
4,***,белорусам,2
...,...,...,...
23224,😘,сдулись,1
23225,😘,сожалению,1
23226,😘,так,1
23227,😘,что,1
